In [33]:
from psaw import PushshiftAPI
import pandas as pd

In [37]:
#create scraping function using pushshiftAPI
def scrape_data(subreddit):
    api = PushshiftAPI()
    subreddit_list = list(api.search_submissions(subreddit=subreddit, filter=['title','subreddit'], limit=10000))
    
    #filter list to show title and subbredit only
    scrape_list = []
    for i in range(len(subreddit_list)):
        scrape_dict = {}
        scrape_dict['subreddit'] = subreddit_list[i][1]
        scrape_dict['title'] = subreddit_list[i][2]
        scrape_list.append(scrape_dict)
    return(scrape_list)

In [41]:
# Call function and create DataFrame
df_nottheonion = pd.DataFrame(scrape_data('nottheonion'))
df_onion = pd.DataFrame(scrape_data('theonion'))


In [42]:
def clean_data(dataframe):

    # Drop duplicate rows
    dataframe.drop_duplicates(subset='title', inplace=True)
    
    # Remove punctation
    dataframe['title'] = dataframe['title'].str.replace('[^\w\s]',' ')

    # Remove numbers 
    dataframe['title'] = dataframe['title'].str.replace('[^A-Za-z]',' ')

    # Make sure any double-spaces are single 
    dataframe['title'] = dataframe['title'].str.replace('  ',' ')
    dataframe['title'] = dataframe['title'].str.replace('  ',' ')

    # Transform all text to lowercase
    dataframe['title'] = dataframe['title'].str.lower()
    
    print("New shape:", dataframe.shape)
    return dataframe.head()


In [43]:
clean_data(df_onion)
clean_data(df_nottheonion)

New shape: (9138, 2)
New shape: (7611, 2)


,subreddit,title
0,nottheonion,zoo may feed animals to animals as funds dry up
1,nottheonion,vince mcmahon is on the council to reopen america
2,nottheonion,donald trump names wwe s vince mcmahon as advi...
3,nottheonion,in war on child porn us turns wounded soldiers...
4,nottheonion,judge asks lawyers to wear shirts get out of b...


In [46]:
# Combine df_onion & df_nottheonion with only 'subreddit' (target) and 'title' (predictor) columns
df = pd.concat([df_onion[['subreddit', 'title']], df_nottheonion[['subreddit', 'title']]])

# Label FakeNews with 1 and RealNews with 0
df["subreddit"] = df["subreddit"].map({"nottheonion": 0, "TheOnion": 1})

#save final dataframe to csv
df.to_csv('./clean_data.csv')